In [9]:
run_dir = '/projects/mmcpnnl/jlee/wrf-mmc/sgp/l2_0805e/'

# Reading data from file1
with open(run_dir+'z_to05-14z/z.e9.d01.QV') as fp:
    data = fp.read()
    
# Reading data from file2
with open(run_dir+'z_to06-00z/z.e9.d01.QV') as fp:
    next(fp)
    data2 = fp.read()
    
# Merging 2 files
# To add the data of file2
# from next line
# data += "\n"
data += data2
  
with open (run_dir+'z_ts/z.e9.d01.QV', 'w') as fp:
    fp.write(data)

In [8]:
run_dir = '/projects/mmcpnnl/jlee/wrf-mmc/sgp/l2_0805e/'
in_file = run_dir+'z_to06-00z/z.e9.d01.QV'
out_file = run_dir+'z_ts/z.e9.d01.QV'

filenames = [in_file, out_file]

with open(out_file, 'w') as outfile:
#     for fname in filenames:
    with open(in_file) as infile:
        next(infile)
        for line in infile:
            outfile.write(line)

In [ ]:
ts_dir = '/projects/mmcpnnl/jlee/wrf-mmc/sgp/l2_0805e/z_to05-14z/'

filenames = ['file1.txt', 'file2.txt', ...]
with open('path/to/output/file', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

In [1]:
import pandas as pd

In [3]:
ts_dir = '/projects/mmcpnnl/jlee/wrf-mmc/sgp/l2_0805e/z_to05-14z/'
ts_file = ts_dir+'z.e12.d01.QV'

df = pd.read_csv(ts_file, skiprows=1, header=None, delim_whitespace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,80
0,0.013889,0.01112,0.01109,0.01105,0.01103,0.01101,0.01099,0.01098,0.01097,0.01096,...,0.00558,0.00517,0.00484,0.00463,0.00484,0.00551,0.00620,0.00689,0.00721,0.00710
1,0.027778,0.01113,0.01110,0.01107,0.01104,0.01101,0.01099,0.01098,0.01097,0.01096,...,0.00560,0.00518,0.00486,0.00465,0.00485,0.00551,0.00620,0.00689,0.00721,0.00711
2,0.041667,0.01114,0.01111,0.01108,0.01105,0.01102,0.01100,0.01098,0.01097,0.01096,...,0.00561,0.00520,0.00488,0.00466,0.00486,0.00552,0.00620,0.00689,0.00721,0.00711
3,0.055556,0.01114,0.01111,0.01109,0.01106,0.01104,0.01101,0.01099,0.01097,0.01096,...,0.00562,0.00522,0.00490,0.00468,0.00487,0.00552,0.00620,0.00689,0.00722,0.00712
4,0.069444,0.01114,0.01112,0.01109,0.01107,0.01105,0.01102,0.01100,0.01098,0.01096,...,0.00564,0.00524,0.00493,0.00470,0.00489,0.00553,0.00621,0.00689,0.00722,0.00712


In [2]:
total_hours = 36
time_step_sec = 50

ideal_file_num = total_hours*60*60/time_step_sec
    
ideal_file_num

21600.0

In [3]:
row num = time_step_sec / (60*60)

0.0016666666666666668

In [37]:
ts_dir = '/projects/mmcpnnl/jlee/wrf-mmc/sgp/sgp_mynn_edmf/analysis/0901e/ts_files_to0901-02/'
ts_file = ts_dir+'s.e32.d02.QV'

In [38]:
ts_file

'/projects/mmcpnnl/jlee/wrf-mmc/sgp/sgp_mynn_edmf/analysis/0901e/ts_files_to0901-02/s.e32.d02.QV'

In [39]:
df2 = pd.read_csv(ts_file, skiprows=1, header=None, delim_whitespace=True)
df2.head()

ParserError: Error tokenizing data. C error: Expected 61 fields in line 426802, saw 427


In [34]:
df2.iloc[0][0][5:13]

'0.001667'

In [36]:
ts_file = ts_dir+'s.c1.d02.TS'
# df = pd.read_csv(ts_file)
df = pd.read_csv(ts_file, skiprows=1, header=None, delim_whitespace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,2,0.001667,1,122,125,297.71671,0.01222,0.04100,6.01960,97647.80469,382.6405,0.0,-23.71324,25.36094,297.30042,301.10989,0.0,0.0,0.0
1,2,0.003333,1,122,125,297.67926,0.01223,0.03354,5.97065,97646.91406,382.6405,0.0,-36.50883,22.03210,296.97849,301.10858,0.0,0.0,0.0
2,2,0.005000,1,122,125,297.65274,0.01223,0.03544,5.90079,97646.63281,382.6405,0.0,-36.26476,22.12527,296.95551,301.10727,0.0,0.0,0.0
3,2,0.006667,1,122,125,297.63370,0.01223,0.03828,5.82131,97646.60156,382.6405,0.0,-35.65777,21.77638,296.93771,301.10596,0.0,0.0,0.0
4,2,0.008333,1,122,125,297.61664,0.01223,0.03951,5.74148,97646.44531,382.6405,0.0,-34.93757,21.31044,296.92010,301.10461,0.0,0.0,0.0


In [35]:
df.iloc[0][0][5:13]

'  0.0016'